In [6]:
from trip_kinematics.HomogenTransformationMatrix import TransformationMatrix
from trip_kinematics.KinematicGroup import KinematicGroup, Transformation
import sympy as sp
from scipy.optimize import minimize

In [7]:
A = Transformation(name='gimbal_joint',values={'rx': 0, 'ry': 0, 'rz': 0}, state_variables=['rx', 'ry', 'rz'])
print(A.state)
A.set_state({"rx" :1})
print(A.state)

{'rx': 0, 'ry': 0, 'rz': 0}
{'rx': 1, 'ry': 0, 'rz': 0}


In [12]:
state=A.get_symbolic_state()
print(state['rx'])
A.set_state(state)
Trafo=A.get_transformation_matrix()

# create function of from symbolic representation
orientation = sp.lambdify([state.values()],Trafo.get_rotation()[0][0])
print(orientation)
x_0 = [0,0,0]
#orientation(x_0)
solution = minimize(orientation,x_0)

print(solution)

rx_{gimbal_joint}
<function _lambdifygenerated at 0x0000023541C2E700>
      fun: 1.0
 hess_inv: array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])
      jac: array([ 0.0000000e+00, -7.4505806e-09, -7.4505806e-09])
  message: 'Optimization terminated successfully.'
     nfev: 4
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([0., 0., 0.])


In [9]:
test = {'A_CSS_P_trans': {}, 'gimbal_joint': {'rx': "bla", 'ry': "blu"}}
sym_test = {'A_CSS_P_trans': {}, 'gimbal_joint': {'rx': "symbol_1", 'ry': "symbol_2"}}
values = []
state_keys= []
for keys in test.keys():
    inner_dictionary = test[keys]
    for inner_keys in inner_dictionary.keys():
        values.append(inner_dictionary[inner_keys])
        state_keys.append([keys,inner_keys])
print(values)
print(state_keys)
print(test['gimbal_joint']['rx'])

['bla', 'blu']
[['gimbal_joint', 'rx'], ['gimbal_joint', 'ry']]
bla


$$\mathtt{\text{[[cos(ry\_\{gimbal\_joint\})*cos(rz\_\{gimbal\_joint\})
  -sin(rz\_\{gimbal\_joint\})*cos(ry\_\{gimbal\_joint\}) sin(ry\_\{gimbal\_joint\}) 0]
 [sin(rx\_\{gimbal\_joint\})*sin(ry\_\{gimbal\_joint\})*cos(rz\_\{gimbal\_joint\}) + sin(rz\_\{gimbal\_joint\})*cos(rx\_\{gimbal\_joint\})
  -sin(rx\_\{gimbal\_joint\})*sin(ry\_\{gimbal\_joint\})*sin(rz\_\{gimbal\_joint\}) + cos(rx\_\{gimbal\_joint\})*cos(rz\_\{gimbal\_joint\})
  -sin(rx\_\{gimbal\_joint\})*cos(ry\_\{gimbal\_joint\}) 0]
 [sin(rx\_\{gimbal\_joint\})*sin(rz\_\{gimbal\_joint\}) - sin(ry\_\{gimbal\_joint\})*cos(rx\_\{gimbal\_joint\})*cos(rz\_\{gimbal\_joint\})
  sin(rx\_\{gimbal\_joint\})*cos(rz\_\{gimbal\_joint\}) + sin(ry\_\{gimbal\_joint\})*sin(rz\_\{gimbal\_joint\})*cos(rx\_\{gimbal\_joint\})
  cos(rx\_\{gimbal\_joint\})*cos(ry\_\{gimbal\_joint\}) 0]
 [0.0 0.0 0.0 1.0]]}}$$

### Idee erlaube transformation selbst symbolische representation zu returnen!

In [24]:
from examples.triped import triped_leg
matrix, symbolic_state_dict = triped_leg.get_symbolic_rep()
forward_kin=matrix.matrix


In [37]:
initial_state               = triped_leg.get_virtual_state()

x_0 = []
state_keys = []
symbols = []
for keys in initial_state.keys():
    inner_dictionary = initial_state[keys]
    inner_symbol = symbolic_state_dict[keys]
    for inner_keys in inner_dictionary.keys():
        x_0.append(inner_dictionary[inner_keys])
        symbols.append(inner_symbol[inner_keys])
        state_keys.append([keys,inner_keys])
print(symbols)
print(state_keys)
solved_states={}
for i in range(len(x_0)):
        if state_keys[i][0] not in solved_states.keys():
            solved_states[state_keys[i][0]] = {}
        solved_states[state_keys[i][0]][state_keys[i][1]]="test"
numeric_forward_kinematics = sp.lambdify([symbols],forward_kin)

[rx_{gimbal_joint}, ry_{gimbal_joint}, rz_{gimbal_joint}, ry_{extend_joint}]
[['gimbal_joint', 'rx'], ['gimbal_joint', 'ry'], ['gimbal_joint', 'rz'], ['extend_joint', 'ry']]


In [34]:
numeric_forward_kinematics(x_0)

[[0.9196879588653216,
  1.2779912894449463e-09,
  -0.39265004561076905,
  0.4381180209976906],
 [-2.8777135393429293e-10, 1.0, 2.580749580071906e-09, -3.559490631896848e-10],
 [0.39265004561076905,
  -2.2604908783910925e-09,
  0.9196879588653216,
  -0.04559136645778717],
 [0, 0, 0, 1.0]]